<a href="https://colab.research.google.com/github/SsekittoIsaac/Freshwater-Fish-Disease-Dataset/blob/main/CONVOLUTION_NEURAL_NETWORK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

# Replace 'your_file.csv' with the path to your CSV file
file_path = '/content/drive/MyDrive/DATA SET/FBS/FBS/SB-Fish-Disease.csv'

# Load data from the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame to verify that the data was loaded correctly
print(df.head())


                         Folder Name  \
0  Bacterial diseases - Aeromoniasis   
1  Bacterial diseases - Aeromoniasis   
2  Bacterial diseases - Aeromoniasis   
3  Bacterial diseases - Aeromoniasis   
4  Bacterial diseases - Aeromoniasis   

                               Image Filename  \
0  Bacterial diseases - Aeromoniasis (1).jpeg   
1   Bacterial diseases - Aeromoniasis (1).jpg   
2   Bacterial diseases - Aeromoniasis (1).png   
3  Bacterial diseases - Aeromoniasis (10).jpg   
4  Bacterial diseases - Aeromoniasis (11).jpg   

                                          Image Path  
0  SB-FishDisease\Bacterial diseases - Aeromonias...  
1  SB-FishDisease\Bacterial diseases - Aeromonias...  
2  SB-FishDisease\Bacterial diseases - Aeromonias...  
3  SB-FishDisease\Bacterial diseases - Aeromonias...  
4  SB-FishDisease\Bacterial diseases - Aeromonias...  


In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Define function to preprocess images
def preprocess_images(image_folder, output_folder, img_size=(100, 100), img_format='png'):
    # Check if the image folder exists
    if not os.path.exists(image_folder):
        raise FileNotFoundError(f"The specified image folder '{image_folder}' does not exist.")

    # Create output folders for train, validation, and test sets
    train_folder = os.path.join(output_folder, 'train')
    val_folder = os.path.join(output_folder, 'validation')
    test_folder = os.path.join(output_folder, 'test')
    for folder in [train_folder, val_folder, test_folder]:
        os.makedirs(folder, exist_ok=True)

    images = []
    labels = []
    train_folders = set()
    val_folders = set()
    test_folders = set()

    for folder_name in os.listdir(image_folder):
        label = folder_name
        folder_path = os.path.join(image_folder, folder_name)
        train_path = os.path.join(train_folder, folder_name)
        val_path = os.path.join(val_folder, folder_name)
        test_path = os.path.join(test_folder, folder_name)
        os.makedirs(train_path, exist_ok=True)
        os.makedirs(val_path, exist_ok=True)
        os.makedirs(test_path, exist_ok=True)

        if np.random.uniform() < 0.8:
            train_folders.add(folder_name)
        elif np.random.uniform() < 0.5:
            val_folders.add(folder_name)
        else:
            test_folders.add(folder_name)

        for img_name in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_name)
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
            img = cv2.resize(img, img_size)  # Resize image
            img = img.astype('float32') / 255.0  # Normalize pixel values
            images.append(img)
            labels.append(label)

            # Save images to respective folders
            if folder_name in train_folders:
                folder = train_path
            elif folder_name in val_folders:
                folder = val_path
            else:
                folder = test_path
            filename = f"{label}_{np.random.randint(1e6)}.{img_format}"
            cv2.imwrite(os.path.join(folder, filename), cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

    print("Data preprocessing complete.")

    return images, labels

# Define CNN architecture
def create_cnn(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),  # Dropout for regularization
        Dense(num_classes, activation='softmax')
    ])
    return model

# Define paths
image_folder = '/content/drive/MyDrive/FBS/SB-FishDisease/SB-FishDisease'
output_folder = '/content/drive/MyDrive/DATA SET'

try:
    # Preprocess images
    images, labels = preprocess_images(image_folder, output_folder)

    # Split data into train, validation, and test sets
    X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)
    X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

    # Convert labels to categorical
    label_encoder = LabelEncoder()
    label_encoder.fit(labels)
    num_classes = len(label_encoder.classes_)
    y_train = to_categorical(label_encoder.transform(y_train), num_classes)
    y_val = to_categorical(label_encoder.transform(y_val), num_classes)
    y_test = to_categorical(label_encoder.transform(y_test), num_classes)

    # Create CNN model
    input_shape = X_train[0].shape
    model = create_cnn(input_shape, num_classes)

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # Learning Rate Reduction
    lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=2, min_lr=0.000001, verbose=1)

    # Fit the model
    history = model.fit(np.array(X_train), np.array(y_train), epochs=20, batch_size=32, validation_data=(np.array(X_val), np.array(y_val)), callbacks=[lr_reduce])

    # Evaluate the model
    loss, accuracy = model.evaluate(np.array(X_test), np.array(y_test))
    print("Test Loss:", loss)
    print("Test Accuracy:", accuracy)
except Exception as e:
    print(e)


Data preprocessing complete.
Epoch 1/20
7/7 [==============================] - 3s 50ms/step - loss: 1.4570 - accuracy: 0.4949 - val_loss: 1.3489 - val_accuracy: 0.5152 - lr: 0.0010
Epoch 2/20
7/7 [==============================] - 0s 20ms/step - loss: 1.3739 - accuracy: 0.5152 - val_loss: 1.3306 - val_accuracy: 0.5152 - lr: 0.0010
Epoch 3/20
7/7 [==============================] - 0s 20ms/step - loss: 1.2956 - accuracy: 0.5253 - val_loss: 1.3591 - val_accuracy: 0.5152 - lr: 0.0010
Epoch 4/20
5/7 [====================>.........] - ETA: 0s - loss: 1.2551 - accuracy: 0.5375
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0003000000142492354.
7/7 [==============================] - 0s 19ms/step - loss: 1.2419 - accuracy: 0.5404 - val_loss: 1.4222 - val_accuracy: 0.5152 - lr: 0.0010
Epoch 5/20
7/7 [==============================] - 0s 19ms/step - loss: 1.2467 - accuracy: 0.5303 - val_loss: 1.2713 - val_accuracy: 0.5455 - lr: 3.0000e-04
Epoch 6/20
7/7 [==============================] - 

In [ ]:
pip install mlflow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.6/147.6 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.9/202.9 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 7.3 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


In [ ]:







import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from datetime import datetime

# Define function to preprocess images
def preprocess_images(image_folder, output_folder, img_size=(100, 100), img_format='png'):
    # Check if the image folder exists
    if not os.path.exists(image_folder):
        raise FileNotFoundError(f"The specified image folder '{image_folder}' does not exist.")

    # Create output folders for train, validation, and test sets
    train_folder = os.path.join(output_folder, 'train')
    val_folder = os.path.join(output_folder, 'validation')
    test_folder = os.path.join(output_folder, 'test')
    for folder in [train_folder, val_folder, test_folder]:
        os.makedirs(folder, exist_ok=True)

    images = []
    labels = []
    train_folders = set()
    val_folders = set()
    test_folders = set()

    for folder_name in os.listdir(image_folder):
        label = folder_name
        folder_path = os.path.join(image_folder, folder_name)
        train_path = os.path.join(train_folder, folder_name)
        val_path = os.path.join(val_folder, folder_name)
        test_path = os.path.join(test_folder, folder_name)
        os.makedirs(train_path, exist_ok=True)
        os.makedirs(val_path, exist_ok=True)
        os.makedirs(test_path, exist_ok=True)

        if np.random.uniform() < 0.8:
            train_folders.add(folder_name)
        elif np.random.uniform() < 0.5:
            val_folders.add(folder_name)
        else:
            test_folders.add(folder_name)

        for img_name in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_name)
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
            img = cv2.resize(img, img_size)  # Resize image
            img = img.astype('float32') / 255.0  # Normalize pixel values
            images.append(img)
            labels.append(label)

            # Save images to respective folders
            if folder_name in train_folders:
                folder = train_path
            elif folder_name in val_folders:
                folder = val_path
            else:
                folder = test_path
            filename = f"{label}_{np.random.randint(1e6)}.{img_format}"
            cv2.imwrite(os.path.join(folder, filename), cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

    print("Data preprocessing complete.")

    return images, labels

# Define CNN architecture
def create_cnn(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),  # Dropout for regularization
        Dense(num_classes, activation='softmax')
    ])
    return model

# Define paths
image_folder = '/content/drive/MyDrive/FBS/SB-FishDisease/SB-FishDisease'
output_folder = '/content/drive/MyDrive/DATA SET'

try:
    # Preprocess images
    images, labels = preprocess_images(image_folder, output_folder)

    # Split data into train, validation, and test sets
    X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)
    X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

    # Convert labels to categorical
    label_encoder = LabelEncoder()
    label_encoder.fit(labels)
    num_classes = len(label_encoder.classes_)
    y_train = to_categorical(label_encoder.transform(y_train), num_classes)
    y_val = to_categorical(label_encoder.transform(y_val), num_classes)
    y_test = to_categorical(label_encoder.transform(y_test), num_classes)

    # Create CNN model
    input_shape = X_train[0].shape
    model = create_cnn(input_shape, num_classes)

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # Learning Rate Reduction
    lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=2, min_lr=0.000001, verbose=1)

    # Fit the model
    history = model.fit(np.array(X_train), np.array(y_train), epochs=20, batch_size=32, validation_data=(np.array(X_val), np.array(y_val)), callbacks=[lr_reduce])

    # Evaluate the model
    loss, accuracy = model.evaluate(np.array(X_test), np.array(y_test))
    print("Test Loss:", loss)
    print("Test Accuracy:", accuracy)

    # Save the model
    save_folder = '/content/drive/MyDrive/SavedModels'
    os.makedirs(save_folder, exist_ok=True)
    model_name = f"fish_disease_model_{datetime.now().strftime('%Y%m%d%H%M%S')}.h5"
    model_path = os.path.join(save_folder, model_name)
    model.save(model_path)
    print(f"Model saved at: {model_path}")
except Exception as e:
    print(e)


Data preprocessing complete.
Epoch 1/20
7/7 [==============================] - 2s 50ms/step - loss: 1.5019 - accuracy: 0.4495 - val_loss: 1.3946 - val_accuracy: 0.5152 - lr: 0.0010
Epoch 2/20
7/7 [==============================] - 0s 21ms/step - loss: 1.3244 - accuracy: 0.5152 - val_loss: 1.3033 - val_accuracy: 0.5152 - lr: 0.0010
Epoch 3/20
7/7 [==============================] - 0s 19ms/step - loss: 1.2548 - accuracy: 0.5152 - val_loss: 1.2978 - val_accuracy: 0.5152 - lr: 0.0010
Epoch 4/20
7/7 [==============================] - 0s 19ms/step - loss: 1.1761 - accuracy: 0.5253 - val_loss: 1.2614 - val_accuracy: 0.5000 - lr: 0.0010
Epoch 5/20
7/7 [==============================] - 0s 19ms/step - loss: 1.1805 - accuracy: 0.6061 - val_loss: 1.2882 - val_accuracy: 0.5152 - lr: 0.0010
Epoch 6/20
5/7 [====================>.........] - ETA: 0s - loss: 1.0745 - accuracy: 0.5813
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0003000000142492354.
7/7 [==============================] - 0s 2

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
!pip install dvc

In [ ]:
!dvc init --no-scm


Initialized DVC repository.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: <https://dvc.org/doc>
- Get help and share ideas: <https://dvc.org/chat>
- Star us on GitHub: <https://github.com/iterative/dvc>


In [ ]:
!dvc remote add -d myremote gdrive:///content/drive/MyDrive/Colab\ Notebooks


Setting 'myremote' as a default remote.


In [ ]:
!dvc add /content/drive/MyDrive/SavedModels/fish_disease_model_20240318131601.h5


⠋ Checking graph
Adding...:   0% 0/1 [00:00<?, ?file/s{'info': ''}]
!
          |0.00 [00:00,     ?file/s]
                                    
!
  0% |          |0/? [00:00<?,    ?files/s]
                                           
  0% 0/1 [00:00<?, ?file/s]
  0% 0/1 [00:00<?, ?file/s{'info': ''}]
                                       
  0% 0/1 [00:00<?, ?files/s]
  0% 0/1 [00:00<?, ?files/s{'info': ''}]
Adding...: 100% 1/1 [00:00<00:00,  1.19file/s{'info': ''}]


In [ ]:
!dvc commit


In [ ]:
!pip install dvc[gdrive]


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 979.2 kB/s eta 0:00:00
  Attempting uninstall: pydrive2
    Found existing installation: PyDrive2 1.6.3
    Uninstalling PyDrive2-1.6.3:
      Successfully uninstalled PyDrive2-1.6.3


In [ ]:
!dvc push


ERROR: unexpected error - 'host'

Having any troubles? Hit us up at https://dvc.org/support, we are always happy to help!


In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from datetime import datetime
import mlflow

# Initialize MLflow
mlflow.set_tracking_uri("file:/content/drive/MyDrive")  # Set your desired tracking URI
mlflow.set_experiment("fish_disease_detection")  # Set the name of your experiment

# Define function to preprocess images
def preprocess_images(image_folder, output_folder, img_size=(100, 100), img_format='png'):
    # Check if the image folder exists
    if not os.path.exists(image_folder):
        raise FileNotFoundError(f"The specified image folder '{image_folder}' does not exist.")

    # Create output folders for train, validation, and test sets
    train_folder = os.path.join(output_folder, 'train')
    val_folder = os.path.join(output_folder, 'validation')
    test_folder = os.path.join(output_folder, 'test')
    for folder in [train_folder, val_folder, test_folder]:
        os.makedirs(folder, exist_ok=True)

    images = []
    labels = []
    train_folders = set()
    val_folders = set()
    test_folders = set()

    for folder_name in os.listdir(image_folder):
        label = folder_name
        folder_path = os.path.join(image_folder, folder_name)
        train_path = os.path.join(train_folder, folder_name)
        val_path = os.path.join(val_folder, folder_name)
        test_path = os.path.join(test_folder, folder_name)
        os.makedirs(train_path, exist_ok=True)
        os.makedirs(val_path, exist_ok=True)
        os.makedirs(test_path, exist_ok=True)

        if np.random.uniform() < 0.8:
            train_folders.add(folder_name)
        elif np.random.uniform() < 0.5:
            val_folders.add(folder_name)
        else:
            test_folders.add(folder_name)

        for img_name in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_name)
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
            img = cv2.resize(img, img_size)  # Resize image
            img = img.astype('float32') / 255.0  # Normalize pixel values
            images.append(img)
            labels.append(label)

            # Save images to respective folders
            if folder_name in train_folders:
                folder = train_path
            elif folder_name in val_folders:
                folder = val_path
            else:
                folder = test_path
            filename = f"{label}_{np.random.randint(1e6)}.{img_format}"
            cv2.imwrite(os.path.join(folder, filename), cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

    print("Data preprocessing complete.")

    return images, labels

# Define CNN architecture
def create_cnn(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),  # Dropout for regularization
        Dense(num_classes, activation='softmax')
    ])
    return model

# Define paths
image_folder = '/content/drive/MyDrive/FBS/SB-FishDisease/SB-FishDisease'
output_folder = '/content/drive/MyDrive/DATA SET'

try:
    # Start MLflow run
    with mlflow.start_run():
        # Preprocess images
        images, labels = preprocess_images(image_folder, output_folder)

        # Split data into train, validation, and test sets
        X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)
        X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

        # Convert labels to categorical
        label_encoder = LabelEncoder()
        label_encoder.fit(labels)
        num_classes = len(label_encoder.classes_)
        y_train = to_categorical(label_encoder.transform(y_train), num_classes)
        y_val = to_categorical(label_encoder.transform(y_val), num_classes)
        y_test = to_categorical(label_encoder.transform(y_test), num_classes)

        # Log parameters
        mlflow.log_param("image_folder", image_folder)
        mlflow.log_param("output_folder", output_folder)
        mlflow.log_param("img_size", (100, 100))
        mlflow.log_param("img_format", 'png')

        # Create CNN model
        input_shape = X_train[0].shape
        model = create_cnn(input_shape, num_classes)

        # Compile the model
        model.compile(optimizer=Adam(learning_rate=0.001),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

        # Learning Rate Reduction
        lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=2, min_lr=0.000001, verbose=1)

        # Fit the model
        history = model.fit(np.array(X_train), np.array(y_train), epochs=20, batch_size=32, validation_data=(np.array(X_val), np.array(y_val)), callbacks=[lr_reduce])

        # Log metrics
        mlflow.log_metric("final_loss", history.history['loss'][-1])
        mlflow.log_metric("final_accuracy", history.history['accuracy'][-1])

        # Evaluate the model
        loss, accuracy = model.evaluate(np.array(X_test), np.array(y_test))
        mlflow.log_metric("test_loss", loss)
        mlflow.log_metric("test_accuracy", accuracy)

        # Save the model
        save_folder = '/content/drive/MyDrive/SavedModels'
        os.makedirs(save_folder, exist_ok=True)
        model_name = f"fish_disease_model_{datetime.now().strftime('%Y%m%d%H%M%S')}.h5"
        model_path = os.path.join(save_folder, model_name)
        model.save(model_path)
        mlflow.log_artifact(model_path)
        mlflow.log_param("model_name", model_name)

        print(f"Model saved at: {model_path}")
except Exception as e:
    print(e)


MlflowException: Invalid experiment ID: 'Colab Notebooks'

In [ ]:







import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from datetime import datetime

# Define function to preprocess images
def preprocess_images(image_folder, output_folder, img_size=(100, 100), img_format='png'):
    # Check if the image folder exists
    if not os.path.exists(image_folder):
        raise FileNotFoundError(f"The specified image folder '{image_folder}' does not exist.")

    # Create output folders for train, validation, and test sets
    train_folder = os.path.join(output_folder, 'train')
    val_folder = os.path.join(output_folder, 'validation')
    test_folder = os.path.join(output_folder, 'test')
    for folder in [train_folder, val_folder, test_folder]:
        os.makedirs(folder, exist_ok=True)

    images = []
    labels = []
    train_folders = set()
    val_folders = set()
    test_folders = set()

    for folder_name in os.listdir(image_folder):
        label = folder_name
        folder_path = os.path.join(image_folder, folder_name)
        train_path = os.path.join(train_folder, folder_name)
        val_path = os.path.join(val_folder, folder_name)
        test_path = os.path.join(test_folder, folder_name)
        os.makedirs(train_path, exist_ok=True)
        os.makedirs(val_path, exist_ok=True)
        os.makedirs(test_path, exist_ok=True)

        if np.random.uniform() < 0.8:
            train_folders.add(folder_name)
        elif np.random.uniform() < 0.5:
            val_folders.add(folder_name)
        else:
            test_folders.add(folder_name)

        for img_name in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_name)
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
            img = cv2.resize(img, img_size)  # Resize image
            img = img.astype('float32') / 255.0  # Normalize pixel values
            images.append(img)
            labels.append(label)

            # Save images to respective folders
            if folder_name in train_folders:
                folder = train_path
            elif folder_name in val_folders:
                folder = val_path
            else:
                folder = test_path
            filename = f"{label}_{np.random.randint(1e6)}.{img_format}"
            cv2.imwrite(os.path.join(folder, filename), cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

    print("Data preprocessing complete.")

    return images, labels

# Define CNN architecture
def create_cnn(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),  # Dropout for regularization
        Dense(num_classes, activation='softmax')
    ])
    return model

# Define paths
image_folder = '/content/drive/MyDrive/FBS/SB-FishDisease/SB-FishDisease'
output_folder = '/content/drive/MyDrive/DATA SET'

try:
    # Preprocess images
    images, labels = preprocess_images(image_folder, output_folder)

    # Split data into train, validation, and test sets
    X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)
    X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

    # Convert labels to categorical
    label_encoder = LabelEncoder()
    label_encoder.fit(labels)
    num_classes = len(label_encoder.classes_)
    y_train = to_categorical(label_encoder.transform(y_train), num_classes)
    y_val = to_categorical(label_encoder.transform(y_val), num_classes)
    y_test = to_categorical(label_encoder.transform(y_test), num_classes)

    # Create CNN model
    input_shape = X_train[0].shape
    model = create_cnn(input_shape, num_classes)

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # Learning Rate Reduction
    lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=2, min_lr=0.000001, verbose=1)

    # Fit the model
    history = model.fit(np.array(X_train), np.array(y_train), epochs=20, batch_size=32, validation_data=(np.array(X_val), np.array(y_val)), callbacks=[lr_reduce])

    # Evaluate the model
    loss, accuracy = model.evaluate(np.array(X_test), np.array(y_test))
    print("Test Loss:", loss)
    print("Test Accuracy:", accuracy)

    # Save the model
    save_folder = '/content/drive/MyDrive/SavedModels'
    os.makedirs(save_folder, exist_ok=True)
    model_name = f"fish_disease_model_{datetime.now().strftime('%Y%m%d%H%M%S')}.h5"
    model_path = os.path.join(save_folder, model_name)
    model.save(model_path)
    print(f"Model saved at: {model_path}")
except Exception as e:
    print(e)


Data preprocessing complete.
Epoch 1/20
7/7 [==============================] - 2s 56ms/step - loss: 1.4348 - accuracy: 0.4495 - val_loss: 1.3393 - val_accuracy: 0.5152 - lr: 0.0010
Epoch 2/20
7/7 [==============================] - 0s 20ms/step - loss: 1.3344 - accuracy: 0.5101 - val_loss: 1.3219 - val_accuracy: 0.5152 - lr: 0.0010
Epoch 3/20
7/7 [==============================] - 0s 20ms/step - loss: 1.2581 - accuracy: 0.5152 - val_loss: 1.3859 - val_accuracy: 0.5152 - lr: 0.0010
Epoch 4/20
7/7 [==============================] - 0s 18ms/step - loss: 1.2789 - accuracy: 0.5303 - val_loss: 1.2548 - val_accuracy: 0.5152 - lr: 0.0010
Epoch 5/20
7/7 [==============================] - 0s 19ms/step - loss: 1.1984 - accuracy: 0.5455 - val_loss: 1.2440 - val_accuracy: 0.5303 - lr: 0.0010
Epoch 6/20
7/7 [==============================] - 0s 19ms/step - loss: 1.1060 - accuracy: 0.5657 - val_loss: 1.2042 - val_accuracy: 0.5758 - lr: 0.0010
Epoch 7/20
7/7 [==============================] - 0s 20ms/s

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


***DOCKER FILE***



In [ ]:
# Use an official Python runtime as a parent image
FROM python:3.8

# Set the working directory in the container
WORKDIR /app

# Copy the current directory contents into the container at /app
COPY . /app

# Install any needed dependencies specified in requirements.txt
RUN pip install --no-cache-dir -r requirements.txt

# Make port 80 available to the world outside this container
EXPOSE 80

# Define environment variable
ENV NAME oyoka

# Run app.py when the container launches
CMD ["python", "app.py"]


In [ ]:
!pip freeze


absl-py==1.4.0
aiohttp==3.9.3
aiohttp-retry==2.8.3
aiosignal==1.3.1
alabaster==0.7.16
albumentations==1.3.1
alembic==1.13.1
altair==4.2.2
amqp==5.2.0
aniso8601==9.0.1
annotated-types==0.6.0
antlr4-python3-runtime==4.9.3
anyio==3.7.1
appdirs==1.4.4
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array-record==0.5.0
arviz==0.15.1
astropy==5.3.4
astunparse==1.6.3
async-timeout==4.0.3
asyncssh==2.14.2
atpublic==4.0
attrs==23.2.0
audioread==3.0.1
autograd==1.6.2
Babel==2.14.0
backcall==0.2.0
beautifulsoup4==4.12.3
bidict==0.23.1
bigframes==0.24.0
billiard==4.2.0
bleach==6.1.0
blinker==1.4
blis==0.7.11
blosc2==2.0.0
bokeh==3.3.4
bqplot==0.12.43
branca==0.7.1
build==1.1.1
CacheControl==0.14.0
cachetools==5.3.3
catalogue==2.0.10
celery==5.3.6
certifi==2024.2.2
cffi==1.16.0
chardet==5.2.0
charset-normalizer==3.3.2
chex==0.1.85
click==8.1.7
click-didyoumean==0.3.0
click-plugins==1.1.1
click-repl==0.3.0
cligj==0.7.2
cloudpathlib==0.16.0
cloudpickle==2.2.1
cmake==3.27.9
cmdstanpy==1.2.1
colorama=

In [ ]:
%%writefile requirements.txt
absl-py==1.4.0
aiohttp==3.9.3
aiohttp-retry==2.8.3
aiosignal==1.3.1
alabaster==0.7.16
albumentations==1.3.1
alembic==1.13.1
altair==4.2.2
amqp==5.2.0
aniso8601==9.0.1
annotated-types==0.6.0
antlr4-python3-runtime==4.9.3
anyio==3.7.1
appdirs==1.4.4
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array-record==0.5.0
arviz==0.15.1
astropy==5.3.4
astunparse==1.6.3
async-timeout==4.0.3
asyncssh==2.14.2
atpublic==4.0
attrs==23.2.0
audioread==3.0.1
autograd==1.6.2
Babel==2.14.0
backcall==0.2.0
beautifulsoup4==4.12.3
bidict==0.23.1
bigframes==0.24.0
billiard==4.2.0
bleach==6.1.0
blinker==1.4
blis==0.7.11
blosc2==2.0.0
bokeh==3.3.4
bqplot==0.12.43
branca==0.7.1
build==1.1.1
CacheControl==0.14.0
cachetools==5.3.3
catalogue==2.0.10
celery==5.3.6
certifi==2024.2.2
cffi==1.16.0
chardet==5.2.0
charset-normalizer==3.3.2
chex==0.1.85
click==8.1.7
click-didyoumean==0.3.0
click-plugins==1.1.1
click-repl==0.3.0
cligj==0.7.2
cloudpathlib==0.16.0
cloudpickle==2.2.1
cmake==3.27.9
cmdstanpy==1.2.1
colorama==0.4.6
colorcet==3.1.0
colorlover==0.3.0
colour==0.1.5
community==1.0.0b1
confection==0.1.4
configobj==5.0.8
cons==0.4.6
contextlib2==21.6.0
contourpy==1.2.0
cryptography==42.0.5
cufflinks==0.17.3
cupy-cuda12x==12.2.0
cvxopt==1.3.2
cvxpy==1.3.3
cycler==0.12.1
cymem==2.0.8
Cython==3.0.9
dask==2023.8.1
datascience==0.17.6
db-dtypes==1.2.0
dbus-python==1.2.18
debugpy==1.6.6
decorator==4.4.2
defusedxml==0.7.1
dictdiffer==0.9.0
diskcache==5.6.3
distributed==2023.8.1
distro==1.7.0
dlib==19.24.2
dm-tree==0.1.8
docker==7.0.0
docutils==0.18.1
dopamine-rl==4.0.6
dpath==2.1.6
duckdb==0.9.2
dulwich==0.21.7
dvc==3.48.4
dvc-data==3.14.1
dvc-gdrive==3.0.1
dvc-http==2.32.0
dvc-objects==5.1.0
dvc-render==1.0.1
dvc-studio-client==0.20.0
dvc-task==0.3.0
earthengine-api==0.1.393
easydict==1.13
ecos==2.0.13
editdistance==0.6.2
eerepr==0.0.4
en-core-web-sm @ https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.7.1/en_core_web_sm-3.7.1-py3-none-any.whl#sha256=86cc141f63942d4b2c5fcee06630fd6f904788d2f0ab005cce45aadb8fb73889
entrypoints==0.4
et-xmlfile==1.1.0
etils==1.7.0
etuples==0.3.9
exceptiongroup==1.2.0
fastai==2.7.14
fastcore==1.5.29
fastdownload==0.0.7
fastjsonschema==2.19.1
fastprogress==1.0.3
fastrlock==0.8.2
filelock==3.13.1
fiona==1.9.6
firebase-admin==5.3.0
Flask==2.2.5
flatbuffers==24.3.7
flatten-dict==0.4.2
flax==0.8.1
flufl.lock==7.1.1
folium==0.14.0
fonttools==4.49.0
frozendict==2.4.0
frozenlist==1.4.1
fsspec==2024.3.0
funcy==2.0
future==0.18.3
gast==0.5.4
gcsfs==2023.6.0
GDAL==3.6.4
gdown==4.7.3
geemap==0.32.0
gensim==4.3.2
geocoder==1.38.1
geographiclib==2.0
geopandas==0.13.2
geopy==2.3.0
gin-config==0.5.0
gitdb==4.0.11
GitPython==3.1.42
glob2==0.7
google==2.0.3
google-ai-generativelanguage==0.4.0
google-api-core==2.11.1
google-api-python-client==2.84.0
google-auth==2.27.0
google-auth-httplib2==0.1.1
google-auth-oauthlib==1.2.0
google-cloud-aiplatform==1.43.0
google-cloud-bigquery==3.12.0
google-cloud-bigquery-connection==1.12.1
google-cloud-bigquery-storage==2.24.0
google-cloud-core==2.3.3
google-cloud-datastore==2.15.2
google-cloud-firestore==2.11.1
google-cloud-functions==1.13.3
google-cloud-iam==2.14.3
google-cloud-language==2.13.3
google-cloud-resource-manager==1.12.3
google-cloud-storage==2.8.0
google-cloud-translate==3.11.3
google-colab @ file:///colabtools/dist/google-colab-1.0.0.tar.gz#sha256=a6dae54fdb980fd35ced7669badeb426dda254ed558842ffad33daa098cdf038
google-crc32c==1.5.0
google-generativeai==0.3.2
google-pasta==0.2.0
google-resumable-media==2.7.0
googleapis-common-protos==1.63.0
googledrivedownloader==0.4
grandalf==0.8
graphene==3.3
graphql-core==3.2.3
graphql-relay==3.2.0
graphviz==0.20.1
greenlet==3.0.3
grpc-google-iam-v1==0.13.0
grpcio==1.62.1
grpcio-status==1.48.2
gspread==3.4.2
gspread-dataframe==3.3.1
gto==1.7.0
gunicorn==21.2.0
gym==0.25.2
gym-notices==0.0.8
h5netcdf==1.3.0
h5py==3.9.0
holidays==0.44
holoviews==1.17.1
html5lib==1.1
httpimport==1.3.1
httplib2==0.22.0
huggingface-hub==0.20.3
humanize==4.7.0
hydra-core==1.3.2
hyperopt==0.2.7
ibis-framework==8.0.0
idna==3.6
imageio==2.31.6
imageio-ffmpeg==0.4.9
imagesize==1.4.1
imbalanced-learn==0.10.1
imgaug==0.4.0
importlib_metadata==7.0.2
importlib_resources==6.3.0
imutils==0.5.4
inflect==7.0.0
iniconfig==2.0.0
intel-openmp==2023.2.4
ipyevents==2.0.2
ipyfilechooser==0.6.0
ipykernel==5.5.6
ipyleaflet==0.18.2
ipython==7.34.0
ipython-genutils==0.2.0
ipython-sql==0.5.0
ipytree==0.2.2
ipywidgets==7.7.1
iterative-telemetry==0.0.8
itsdangerous==2.1.2
jax==0.4.23
jaxlib @ https://storage.googleapis.com/jax-releases/cuda12/jaxlib-0.4.23+cuda12.cudnn89-cp310-cp310-manylinux2014_x86_64.whl#sha256=8e42000672599e7ec0ea7f551acfcc95dcdd0e22b05a1d1f12f97b56a9fce4a8
jeepney==0.7.1
jieba==0.42.1
Jinja2==3.1.3
joblib==1.3.2
jsonpickle==3.0.3
jsonschema==4.19.2
jsonschema-specifications==2023.12.1
jupyter-client==6.1.12
jupyter-console==6.1.0
jupyter-server==1.24.0
jupyter_core==5.7.2
jupyterlab_pygments==0.3.0
jupyterlab_widgets==3.0.10
kaggle==1.5.16
kagglehub==0.2.0
keras==2.15.0
keyring==23.5.0
kiwisolver==1.4.5
kombu==5.3.5
langcodes==3.3.0
launchpadlib==1.10.16
lazr.restfulclient==0.14.4
lazr.uri==1.0.6
lazy_loader==0.3
libclang==16.0.6
librosa==0.10.1
lightgbm==4.1.0
linkify-it-py==2.0.3
llvmlite==0.41.1
locket==1.0.0
logical-unification==0.4.6
lxml==4.9.4
Mako==1.3.2
malloy==2023.1067
Markdown==3.5.2
markdown-it-py==3.0.0
MarkupSafe==2.1.5
matplotlib==3.7.1
matplotlib-inline==0.1.6
matplotlib-venn==0.11.10
mdit-py-plugins==0.4.0
mdurl==0.1.2
miniKanren==1.0.3
missingno==0.5.2
mistune==0.8.4
mizani==0.9.3
mkl==2023.2.0
ml-dtypes==0.2.0
mlflow==2.11.1
mlxtend==0.22.0
more-itertools==10.1.0
moviepy==1.0.3
mpmath==1.3.0
msgpack==1.0.8
multidict==6.0.5
multipledispatch==1.0.0
multitasking==0.0.11
murmurhash==1.0.10
music21==9.1.0
natsort==8.4.0
nbclassic==1.0.0
nbclient==0.10.0
nbconvert==6.5.4
nbformat==5.10.2
nest-asyncio==1.6.0
networkx==3.2.1
nibabel==4.0.2
nltk==3.8.1
notebook==6.5.5
notebook_shim==0.2.4
numba==0.58.1
numexpr==2.9.0
numpy==1.25.2
oauth2client==4.1.3
oauthlib==3.2.2
omegaconf==2.3.0
opencv-contrib-python==4.8.0.76
opencv-python==4.8.0.76
opencv-python-headless==4.9.0.80
openpyxl==3.1.2
opt-einsum==3.3.0
optax==0.2.1
orbax-checkpoint==0.4.4
orjson==3.9.15
osqp==0.6.2.post8
packaging==23.2
pandas==1.5.3
pandas-datareader==0.10.0
pandas-gbq==0.19.2
pandas-stubs==1.5.3.230304
pandocfilters==1.5.1
panel==1.3.8
param==2.0.2
parso==0.8.3
parsy==2.1
partd==1.4.1
pathlib==1.0.1
pathspec==0.12.1
patsy==0.5.6
peewee==3.17.1
pexpect==4.9.0
pickleshare==0.7.5
Pillow==9.4.0
pip-tools==6.13.0
platformdirs==3.11.0
plotly==5.15.0
plotnine==0.12.4
pluggy==1.4.0
polars==0.20.2
pooch==1.8.1
portpicker==1.5.2
prefetch-generator==1.0.3
preshed==3.0.9
prettytable==3.10.0
proglog==0.1.10
progressbar2==4.2.0
prometheus_client==0.20.0
promise==2.3
prompt-toolkit==3.0.43
prophet==1.1.5
proto-plus==1.23.0
protobuf==3.20.3
psutil==5.9.5
psycopg2==2.9.9
ptyprocess==0.7.0
py-cpuinfo==9.0.0
py4j==0.10.9.7
pyarrow==14.0.2
pyarrow-hotfix==0.6
pyasn1==0.5.1
pyasn1-modules==0.3.0
pycocotools==2.0.7
pycparser==2.21
pydantic==2.6.4
pydantic_core==2.16.3
pydata-google-auth==1.8.2
pydot==1.4.2
pydot-ng==2.0.0
pydotplus==2.0.2
PyDrive==1.3.1
PyDrive2==1.19.0
pyerfa==2.0.1.1
pygame==2.5.2
pygit2==1.14.1
Pygments==2.16.1
PyGObject==3.42.1
pygtrie==2.5.0
PyJWT==2.3.0
pymc==5.10.4
pymystem3==0.2.0
PyOpenGL==3.1.7
pyOpenSSL==24.1.0
pyparsing==3.1.2
pyperclip==1.8.2
pyproj==3.6.1
pyproject_hooks==1.0.0
pyshp==2.3.1
PySocks==1.7.1
pytensor==2.18.6
pytest==7.4.4
python-apt @ file:///backend-container/containers/python_apt-0.0.0-cp310-cp310-linux_x86_64.whl#sha256=b209c7165d6061963abe611492f8c91c3bcef4b7a6600f966bab58900c63fefa
python-box==7.1.1
python-dateutil==2.8.2
python-louvain==0.16
python-slugify==8.0.4
python-utils==3.8.2
pytz==2023.4
pyviz_comms==3.0.1
PyWavelets==1.5.0
PyYAML==6.0.1
pyzmq==23.2.1
qdldl==0.1.7.post0
qudida==0.0.4
querystring-parser==1.2.4
ratelim==0.1.6
referencing==0.33.0
regex==2023.12.25
requests==2.31.0
requests-oauthlib==1.4.0
requirements-parser==0.5.0
rich==13.7.1
rpds-py==0.18.0
rpy2==3.4.2
rsa==4.9
ruamel.yaml==0.18.6
ruamel.yaml.clib==0.2.8
safetensors==0.4.2
scikit-image==0.19.3
scikit-learn==1.2.2
scipy==1.11.4
scmrepo==3.3.0
scooby==0.9.2
scs==3.2.4.post1
seaborn==0.13.1
SecretStorage==3.3.1
semver==3.0.2
Send2Trash==1.8.2
sentencepiece==0.1.99
shapely==2.0.3
shortuuid==1.0.13
shtab==1.7.1
six==1.16.0
sklearn-pandas==2.2.0
smart-open==6.4.0
smmap==5.0.1
sniffio==1.3.1
snowballstemmer==2.2.0
sortedcontainers==2.4.0
soundfile==0.12.1
soupsieve==2.5
soxr==0.3.7
spacy==3.7.4
spacy-legacy==3.0.12
spacy-loggers==1.0.5
Sphinx==5.0.2
sphinxcontrib-applehelp==1.0.8
sphinxcontrib-devhelp==1.0.6
sphinxcontrib-htmlhelp==2.0.5
sphinxcontrib-jsmath==1.0.1
sphinxcontrib-qthelp==1.0.7
sphinxcontrib-serializinghtml==1.1.10
SQLAlchemy==2.0.28
sqlglot==20.11.0
sqlparse==0.4.4
sqltrie==0.11.0
srsly==2.4.8
stanio==0.3.0
statsmodels==0.14.1
sympy==1.12
tables==3.8.0
tabulate==0.9.0
tbb==2021.11.0
tblib==3.0.0
tenacity==8.2.3
tensorboard==2.15.2
tensorboard-data-server==0.7.2
tensorflow==2.15.0
tensorflow-datasets==4.9.4
tensorflow-estimator==2.15.0
tensorflow-gcs-config==2.15.0
tensorflow-hub==0.16.1
tensorflow-io-gcs-filesystem==0.36.0
tensorflow-metadata==1.14.0
tensorflow-probability==0.23.0
tensorstore==0.1.45
termcolor==2.4.0
terminado==0.18.1
text-unidecode==1.3
textblob==0.17.1
tf-slim==1.1.0
tf_keras==2.15.1
thinc==8.2.3
threadpoolctl==3.3.0
tifffile==2024.2.12
tinycss2==1.2.1
tokenizers==0.15.2
toml==0.10.2
tomli==2.0.1
tomlkit==0.12.4
toolz==0.12.1
torch @ https://download.pytorch.org/whl/cu121/torch-2.2.1%2Bcu121-cp310-cp310-linux_x86_64.whl#sha256=1adf430f01ff649c848ac021785e18007b0714fdde68e4e65bd0c640bf3fb8e1
torchaudio @ https://download.pytorch.org/whl/cu121/torchaudio-2.2.1%2Bcu121-cp310-cp310-linux_x86_64.whl#sha256=23f6236429e2bf676b820e8e7221a1d58aaf908bff2ba2665aa852df71a97961
torchdata==0.7.1
torchsummary==1.5.1
torchtext==0.17.1
torchvision @ https://download.pytorch.org/whl/cu121/torchvision-0.17.1%2Bcu121-cp310-cp310-linux_x86_64.whl#sha256=27af47915f6e762c1d44e58e8088d22ac97445668f9f793524032b2baf4f34bd
tornado==6.3.3
tqdm==4.66.2
traitlets==5.7.1
traittypes==0.2.1
transformers==4.38.2
triton==2.2.0
tweepy==4.14.0
typer==0.9.0
types-pytz==2024.1.0.20240203
types-setuptools==69.1.0.20240310
typing_extensions==4.10.0
tzdata==2024.1
tzlocal==5.2
uc-micro-py==1.0.3
uritemplate==4.1.1
urllib3==2.0.7
vega-datasets==0.9.0
vine==5.1.0
voluptuous==0.14.2
wadllib==1.3.6
wasabi==1.1.2
wcwidth==0.2.13
weasel==0.3.4
webcolors==1.13
webencodings==0.5.1
websocket-client==1.7.0
Werkzeug==3.0.1
widgetsnbextension==3.6.6
wordcloud==1.9.3
wrapt==1.14.1
xarray==2023.7.0
xarray-einstats==0.7.0
xgboost==2.0.3
xlrd==2.0.1
xyzservices==2023.10.1
yarl==1.9.4
yellowbrick==1.5
yfinance==0.2.37
zc.lockfile==3.0.post1
zict==3.0.0
zipp==3.18.0



Writing requirements.txt


In [ ]:
import os

# Get the current working directory
cwd = os.getcwd()

# List files in the current directory
files = os.listdir(cwd)

# Print the current working directory and files
print("Current Working Directory:", cwd)
print("Files in the directory:", files)


Current Working Directory: /content
Files in the directory: ['.config', 'drive', 'requirements.txt', '.dvc', '.dvcignore', 'sample_data']
